In [1]:
import oss2
import json
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import requests
with open("../aliyun_key.json" ,'r') as key:
    AccessKeyDict=json.loads(key.read())
AccessKeyId=AccessKeyDict["AccessKeyId"]
AccessKeySecret=AccessKeyDict["AccessKeySecret"]
auth = oss2.Auth(AccessKeyId, AccessKeySecret)
bucket = oss2.Bucket(auth, 'https://oss-cn-shanghai.aliyuncs.com', 'ocr-yjs')
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkcore.auth.credentials import AccessKeyCredential
from aliyunsdkcore.auth.credentials import StsTokenCredential
credentials = AccessKeyCredential(AccessKeyId, AccessKeySecret)
client = AcsClient(region_id='cn-shanghai', credential=credentials)
import seaborn as sns

In [2]:
"""
https://help.aliyun.com/document_detail/203509.html
人像素描风格化
人像素描风格化可以对一张人物图像自动裁剪其头部区域，生成大头照下的素描画效果。
"""

from aliyunsdkfacebody.request.v20191230.GenerateHumanSketchStyleRequest import GenerateHumanSketchStyleRequest

request = GenerateHumanSketchStyleRequest()
request.set_accept_format('json')

filepath="images/test2.jpg"
with open(filepath, 'rb') as fileobj:
    bucket.put_object(filepath, fileobj)
url="https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/"+filepath
print(url)

request.set_ImageURL(url)
request.set_ReturnType("full")


response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)
ImageURL=response["Data"]["ImageURL"]

https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/images/test2.jpg
{'RequestId': 'A42FC644-0872-5A75-AB8D-5B82DA3C92A2', 'Data': {'ImageURL': 'http://vibktprfx-prod-prod-aic-gd-cn-shanghai.oss-cn-shanghai.aliyuncs.com/person-image-pencil/2a576735-93b8-4d8b-981d-0738b0b63bb3_24a7_20210821-014257.png?Expires=1629511977&OSSAccessKeyId=LTAI4FoLmvQ9urWXgSRpDvh1&Signature=vrpWk08uEoHDtCv07UAJ7LFPYAw%3D'}}


In [3]:
"""
ImageURL	String	是	http://invi-label.oss-cn-shanghai.aliyuncs.com/label/temp/cartoon/test_data/g0.jpg	
图像URL地址。当前仅支持上海地域的OSS链接，如何生成URL请参见生成URL。

ReturnType	String	否	head	
图像返回形式。取值如下：

head：大头照（默认）
full：全身照
推荐使用大头照，处理效果更佳。
"""

'\nMakeupType\tString\t是\twhole\t\n美妆类型，当前支持whole（整妆）。\n\nResourceType\tString\t是\t0\t\n美妆使用的风格，具体包括：0（whole）、1（基础妆）、2（少女妆）、3（活力妆）、4（优雅妆）、5（魅惑妆）、6（梅子妆）。\n\nStrength\tFloat\t是\t1\t\n美妆强度，取值范围0～1。\n'

In [3]:
with open("images/人像素描风格化后.jpg",'wb') as file:
    file.write(requests.get(ImageURL).content)